In [13]:
import PyKEP as pkp
from PyKEP import epoch, AU, MU_SUN, DEG2RAD, lambert_problem, DAY2SEC, DAY2YEAR
from PyKEP.orbit_plots import plot_planet, plot_lambert
import numpy as np
from MySateliteSim import MySateliteSim
import matplotlib as mpl
import matplotlib.pyplot as plt




system = MySateliteSim(87464)
def init_planet(planet):
    x0 = system.x0[planet]
    y0 = system.y0[planet]

    e = system.e[planet]
    a = system.a[planet]
    b = a*np.sqrt(1-e**2)
    M = np.arctan(x0*b/(y0*a)) - e*x0/a
    i = 0.00001
    W = 0.00001
    w = system.psi[planet]
    mu_star = system.starMass * MU_SUN
    mu_planet = system.mass[planet] * MU_SUN
    radius = system.radius[planet] * 1000
    safe_radius = radius + 100000
    return pkp.planet.keplerian(epoch(0), np.array((a*AU, e, i, W, w, M)), mu_star, mu_planet, radius, safe_radius, 'home')
#home = init_planet(0)
#a is the semi-major axis, 
#e the eccentriciy, 
#i the inclination, 
#W the Right Axcension of the Acending Node, 
#w the argument of perigee 
#M the mean anomaly 



In [18]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
axis = fig.gca(projection = "3d")

t1 = epoch(14.32/DAY2YEAR)
t2 = epoch(17.32/DAY2YEAR)
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC

planet1 = init_planet(0)
plot_planet(planet1, t0=t1, legend=True, units = AU, ax = axis)
planet4 = init_planet(4)
plot_planet(planet4, t0=t2, legend=True, units = AU, ax = axis)

rH, vH = planet1.eph(t1)
rT, vT = planet4.eph(t2)

mu_star = system.starMass * MU_SUN
l = lambert_problem(rH, rT, dt, mu_star)
plot_lambert(l, legend=True, units = AU, ax = axis)
plt.show()

In [30]:
print system.psi
print system.omega
planet = 1
psi0 = system.psi[planet]; omega0 = system.omega[planet]
true_anomaly = system.omega - system.psi
true_anomaly[true_anomaly < 0] += 2*np.pi
print true_anomaly
r = np.array((np.cos(psi0), np.sin(psi0)))
r2 = np.array((np.cos(omega0),np.sin(omega0)))
pos3 = np.array((system.x0,system.y0)).T
r3 = pos3[planet]/np.linalg.norm(pos3[planet])              

y_o = system.y0[planet] /np.linalg.norm(system.y0[planet])

plt.plot((0,r[0]),(0,r[1]))
plt.plot((0,r2[0]),(0,r2[1]))
plt.scatter((0,r3[0]),(0,r3[1]))
plt.legend(['psi','omega'])
plt.axis('equal')
plt.show()

[ 0.          0.04554273  1.79075859  3.44648978  2.13146297  5.27493537
  4.41729641]
[ 0.          5.31983772  0.91036524  1.65848031  0.75800604  0.29551564
  2.07347334]
[ 0.          5.27429498  5.40279196  4.49517584  4.90972837  1.30376557
  3.93936224]
